<a href="https://colab.research.google.com/github/viincci/viincci/blob/main/.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[tourch] datasets sentencepiece

In [2]:
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd


In [4]:
recipe = pd.read_csv(open("recipe.csv","rb")).drop(['url',  'author', 'rating','rating_count', 'review_count', 'prep_time', 'cook_time',  'servings', 'calories', 'carbohydrates_g', 'sugars_g', 'fat_g', 'saturated_fat_g','cholesterol_mg', 'protein_g', 'dietary_fiber_g', 'sodium_mg','calories_from_fat', 'calcium_mg', 'iron_mg', 'magnesium_mg','potassium_mg', 'zinc_mg', 'phosphorus_mg', 'vitamin_a_iu_IU','niacin_equivalents_mg', 'vitamin_b6_mg', 'vitamin_c_mg', 'folate_mcg','thiamin_mg', 'riboflavin_mg', 'vitamin_e_iu_IU', 'vitamin_k_mcg','biotin_mcg', 'vitamin_b12_mcg', 'mono_fat_g', 'poly_fat_g','trans_fatty_acid_g', 'omega_3_fatty_acid_g', 'omega_6_fatty_acid_g','instructions_list', 'image'], axis=1).to_dict()




In [5]:
def clean_data(instructions,responses):
    lst = {}
    for i , k in enumerate(instructions):
        if k in lst.keys():
            lst[f'{k}'].append(i)
        elif k not in lst.keys():
            lst[k] = [i]
    newdct = {}
    for k , v in lst.items():
        anw = ""
        for i in v:
            anw += responses[i]
        newdct[k] = anw
    # print(f"{[x for x in newdct.keys()][-1]} : {newdct.get([x for x in lst.keys()][-1])}")
    return newdct

def getdata():
    data = {}
    # for i,k in clean_data(dataset['validation']['question'],dataset['validation']['answer']).items():
    #     data[i] = k
    # sem = json.load(open('datasets/sem01.json','r'))
    # for i,k in sem.items():
    #     data[i] = k

    _dct = []
    # data = {}
    for k,v in recipe['title'].items():
        name = v
        make = f"{v} is a {recipe['category'].get(k)} \n {recipe['description'].get(k)} it take {recipe['total_time'].get(k)} to cook this meal \n this meal yields {recipe['yields'].get(k)} \n Ingredients {recipe['ingredients'].get(k)} \n Directions {recipe['directions'].get(k)}  "
        data[name] = make
    for k, v in data.items():
        _dct.append({"Question": k, "Answer" : v })

    # Combine the datasets into a single DataFrame
    dataframe = pd.json_normalize(_dct)
    # json.dump(data', 'pen('datasets/sem01.json','w'))
    return dataframe

In [ ]:
path = "google/flan-t5-small"
model_name = f"{path}"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# Preprocess and tokenize the dataset
def tokenize_batch(batch):
    source_texts = batch["Question"]
    target_texts = batch["Answer"]
    tokenized_source = tokenizer.batch_encode_plus(
        source_texts,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt",
        max_length=128,
    )
    tokenized_target = tokenizer.batch_encode_plus(
        target_texts,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt",
        max_length=128,
    )
    return {
        "input_ids": tokenized_source["input_ids"],
        "attention_mask": tokenized_source["attention_mask"],
        "labels": tokenized_target["input_ids"],
    }



In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output_dir",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
)


In [ ]:
combined_dataset = Dataset.from_pandas(getdata()).train_test_split(shuffle=True)
# Assuming "train" is the split name for training data in the wiki_qa dataset
train_dataset = combined_dataset["train"].map(tokenize_batch, batched=True,batch_size=1500)
eval_dataset = combined_dataset["test"].map(tokenize_batch, batched=True,batch_size=1500)  # Create an evaluation dataset


In [ ]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Provide the evaluation dataset
)


In [ ]:

# Start fine-tuning
trainer.train()


In [ ]:
# Save the trained model
model.save_pretrained("./ChefSemblanceFlanT5Base")
tokenizer.save_pretrained("./ChefSemblanceFlanT5Base")